In [2]:
#based on http://www.jw.pe/blog/post/quantifying-sufjan-stevens-with-the-genius-api-and-nltk/
import requests
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import clear_output

In [9]:
CLIENT_ACCESS_TOKEN = "lTv0ZCyw1mkXQ7_9bZ9GmEdc9vPh2_VQzhFCpERSOYiFA1fXjEVrI8BY-D9_k6E2"
BASE_URI = "https://api.genius.com"
DREXLER_ARTIST_ID = 344463

In [85]:
def _get(path, params=None, headers=None):

    url = '/'.join([BASE_URI, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}
    response = requests.get(url=url, params=params, headers=headers)
    response.raise_for_status()

    return response.json()


def get_artist_songs(artist_id):

    current_page = 1
    next_page = True
    songs = []

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}
        data = _get(path=path, params=params)
        page_songs = data['response']['songs']
        if page_songs:
            songs += page_songs
            current_page += 1
        else:
            next_page = False

    return songs


In [4]:
songs = get_artist_songs(DREXLER_ARTIST_ID)
songs = [song for song in songs if song['primary_artist']['id'] == DREXLER_ARTIST_ID]

songs

[{'annotation_count': 1,
  'api_path': '/songs/1427096',
  'full_title': '12 Segundos De Oscuridad by\xa0Jorge\xa0Drexler',
  'header_image_thumbnail_url': 'https://images.genius.com/55f0b53f0d9ea3d5252558200048c917.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/55f0b53f0d9ea3d5252558200048c917.640x640x1.jpg',
  'id': 1427096,
  'lyrics_owner_id': 5133257,
  'lyrics_state': 'complete',
  'path': '/Jorge-drexler-12-segundos-de-oscuridad-lyrics',
  'primary_artist': {'api_path': '/artists/344463',
   'header_image_url': 'https://images.genius.com/d55c4797f307cd736c2f3fca5e7cd896.750x1000x1.jpg',
   'id': 344463,
   'image_url': 'https://images.genius.com/64877490f08e60dbb4e729e948f24413.1000x1000x1.jpg',
   'is_meme_verified': False,
   'is_verified': False,
   'name': 'Jorge Drexler',
   'url': 'https://genius.com/artists/Jorge-drexler'},
  'pyongs_count': None,
  'song_art_image_thumbnail_url': 'https://images.genius.com/55f0b53f0d9ea3d5252558200048c917.300x300x1.jpg'

In [20]:
def scrape_lyrics(url):

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    lyrics = soup.find("div", {"class": "lyrics"})
    lyrics = cleanhtml(str(lyrics))
    return lyrics

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


for song in songs:
    url = song['url']
    lyrics = scrape_lyrics(url)
    song['lyrics'] = lyrics

In [22]:
df = pd.read_json(json.dumps(songs))
df.to_csv('data/raw_songs.csv', sep=';',index=False)

In [72]:
df = pd.read_csv('data/raw_songs.csv')

### Let's clean the db a little bit

In [73]:
df.head()

,annotation_count,api_path,full_title,header_image_thumbnail_url,header_image_url,id,lyrics,lyrics_owner_id,lyrics_state,path,primary_artist,pyongs_count,song_art_image_thumbnail_url,stats,title,title_with_featured,url
0,1,/songs/1427096,12 Segundos De Oscuridad by Jorge Drexler,https://images.genius.com/55f0b53f0d9ea3d52525...,https://images.genius.com/55f0b53f0d9ea3d52525...,1427096,\n\nGira el haz de luz para que se vea desde a...,5133257,complete,/Jorge-drexler-12-segundos-de-oscuridad-lyrics,"{'api_path': '/artists/344463', 'header_image_...",NaN,https://images.genius.com/55f0b53f0d9ea3d52525...,"{'hot': False, 'unreviewed_annotations': 0}",12 Segundos De Oscuridad,12 Segundos De Oscuridad,https://genius.com/Jorge-drexler-12-segundos-d...
1,1,/songs/1276296,730 días by Jorge Drexler,https://images.genius.com/ae231f4aaad30926cf47...,https://images.genius.com/ae231f4aaad30926cf47...,1276296,\n\nNo hay rincón en esta casa\nQue no te haga...,1549345,complete,/Jorge-drexler-730-dias-lyrics,"{'api_path': '/artists/344463', 'header_image_...",NaN,https://images.genius.com/ae231f4aaad30926cf47...,"{'hot': False, 'unreviewed_annotations': 0}",730 días,730 días,https://genius.com/Jorge-drexler-730-dias-lyrics
2,1,/songs/2004658,730 dias - Cara B by Jorge Drexler,https://images.genius.com/64877490f08e60dbb4e7...,https://images.genius.com/64877490f08e60dbb4e7...,2004658,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,1549345,complete,/Jorge-drexler-730-dias-cara-b-lyrics,"{'api_path': '/artists/344463', 'header_image_...",NaN,https://images.genius.com/64877490f08e60dbb4e7...,"{'hot': False, 'unreviewed_annotations': 0}",730 dias - Cara B,730 dias - Cara B,https://genius.com/Jorge-drexler-730-dias-cara...
3,0,/songs/3204468,Abracadabras by Jorge Drexler (Ft. Julieta Ven...,https://images.genius.com/738c67c5bf2a2165ae26...,https://images.genius.com/738c67c5bf2a2165ae26...,3204468,"\n\n\n Letra de ""Abracadabras"" ft. Julieta Ven...",3848985,complete,/Jorge-drexler-abracadabras-lyrics,"{'api_path': '/artists/344463', 'header_image_...",NaN,https://images.genius.com/738c67c5bf2a2165ae26...,"{'hot': False, 'unreviewed_annotations': 0}",Abracadabras,Abracadabras (Ft. Julieta Venegas),https://genius.com/Jorge-drexler-abracadabras-...
4,1,/songs/1719443,Al Otro Lado del Río by Jorge Drexler,https://images.genius.com/d921a9bb71b867aa1470...,https://images.genius.com/d921a9bb71b867aa1470...,1719443,"\n\n\nLetra de ""Al Otro Lado del Río""Clavo mi ...",3848985,complete,/Jorge-drexler-al-otro-lado-del-rio-lyrics,"{'api_path': '/artists/344463', 'header_image_...",NaN,https://images.genius.com/d921a9bb71b867aa1470...,"{'hot': False, 'unreviewed_annotations': 1}",Al Otro Lado del Río,Al Otro Lado del Río,https://genius.com/Jorge-drexler-al-otro-lado-...


In [74]:
del df['annotation_count']
del df['api_path']
del df['full_title']
del df['header_image_thumbnail_url']
del df['header_image_url']
del df['lyrics_owner_id']
del df['path']
del df['primary_artist']
del df['pyongs_count']
del df['song_art_image_thumbnail_url']
del df['stats']
#wow, almost all information is useless
del df['title_with_featured']

#### Not really sure what the 'complete' lyrics_state means, so let's do exploration

In [75]:
df[df['lyrics_state'] != 'complete']

,id,lyrics,lyrics_state,title,url
54,3157499,\n \n Lyrics for this song...,unreleased,El Valle de Las Leñas Amarillas,https://genius.com/Jorge-drexler-el-valle-de-l...


In [76]:
df.loc[54].lyrics

'\n          \n            Lyrics for this song have yet to be released. Please check back once the song has been released.\n          \n        '

#### Well, that makes sense. It's his only instrumental song. We'll take it from the dataset since it useless to us.

In [77]:
del df['lyrics_state']
df = df.drop(54)

#### There's some songs in portuguese which are oddly duplicates!!!!

In [78]:
df = df.drop(df[df['title'].str.contains("tradução")].index)

In [79]:
df

,id,lyrics,title,url
0,1427096,\n\nGira el haz de luz para que se vea desde a...,12 Segundos De Oscuridad,https://genius.com/Jorge-drexler-12-segundos-d...
1,1276296,\n\nNo hay rincón en esta casa\nQue no te haga...,730 días,https://genius.com/Jorge-drexler-730-dias-lyrics
2,2004658,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,730 dias - Cara B,https://genius.com/Jorge-drexler-730-dias-cara...
3,3204468,"\n\n\n Letra de ""Abracadabras"" ft. Julieta Ven...",Abracadabras,https://genius.com/Jorge-drexler-abracadabras-...
4,1719443,"\n\n\nLetra de ""Al Otro Lado del Río""Clavo mi ...",Al Otro Lado del Río,https://genius.com/Jorge-drexler-al-otro-lado-...
5,1092085,\n\nMira\nYo aquí me bajo\nYo dejo el tren en ...,Alto el fuego,https://genius.com/Jorge-drexler-alto-el-fuego...
6,1251195,\n\nAna\nQué no daría yo por ver un día\nTu ro...,Ana,https://genius.com/Jorge-drexler-ana-lyrics
7,1331843,\n\nAnda\n\nQuítate el vestido\nLas flores y l...,Anda,https://genius.com/Jorge-drexler-anda-lyrics
8,1375601,\n\nAntes de mi tu no eras tu\nAntes de ti yo ...,Antes,https://genius.com/Jorge-drexler-antes-lyrics
9,1360422,\n\nEra el tiempo del cambio\nEl tiempo de la ...,Aquellos tiempos,https://genius.com/Jorge-drexler-aquellos-tiem...


#### Ok now, we have an album called Cara B which has live versions, covers and other stuff. Luckily, all the songs end in '- Cara B'
#### Only original songs in it not featured on other albums:
- Gracias 
- Dove Sei?
- Fractura de escafoides tarsiano derecho (the lyrics are just a print out from his medical report, not valid)

In [80]:
df[df['title'].str.contains("- Cara B")]

,id,lyrics,title,url
2,2004658,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,730 dias - Cara B,https://genius.com/Jorge-drexler-730-dias-cara...
22,1956248,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Club tonight - Cara B,https://genius.com/Jorge-drexler-club-tonight-...
27,2006543,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,"Dance, dance, dance! - Cara B",https://genius.com/Jorge-drexler-dance-dance-d...
29,1938025,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Dance me to the end of love - Cara B,https://genius.com/Jorge-drexler-dance-me-to-t...
33,1970248,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Deseo - Cara B,https://genius.com/Jorge-drexler-deseo-cara-b-...
36,1994931,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Disneylandia - Cara B,https://genius.com/Jorge-drexler-disneylandia-...
38,1937124,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Dom de iludir - Cara B,https://genius.com/Jorge-drexler-dom-de-iludir...
42,1936427,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Dove sei - Cara B,https://genius.com/Jorge-drexler-dove-sei-cara...
56,1946842,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Equipaje - Cara B,https://genius.com/Jorge-drexler-equipaje-cara...
62,1976094,\n\nSi cuando vuelves a casa\nMe ves sombrío\n...,Flores en el mar - Cara B,https://genius.com/Jorge-drexler-flores-en-el-...


#### Some observations:
- Apparently all lyrics from that album are fucked up. The system confused it with the actual song 'Cara B'. Luckily we don't mind that much. We'll have to fix the songs we need by hand.
- Two songs are missing: Cara B y La Vida es más compleja de lo que parece.

### Let's drop those songs and fix the lyrics we care about

In [81]:
indexes_cara_b = df[df['title'].str.contains("- Cara B")].index
indexes_cara_b = indexes_cara_b.drop([70])
df = df.drop(indexes_cara_b)

df.at[70, 'lyrics'] = """Gracias, gracias, gracias
Quiero agradecer a quien corresponda
Why a nosotros dos el premio a la audacia
Por entrar al mar por la parte más honda
Tiempo al tiempo al tiempo
Cada huella irá encontrando su arena
Cada beso deteniendo un momento
Why cada canción matando una pena
Yo quiero dar la noticia
Yo quiero gritar que me quieres a los cuatro vientos
No quiero guardarme lo que siento
Los relojes se detuvieron en aquel corredor vacío
Yo te vi llegar flotando a un palmo del suelo
Tus labios rumbo a los míos
Nunca digas nunca
Todo pasa en el momento menos pensado
No pensé que fuera a oir tu pregunta
Ni a tu corazón latiendo a mi lado
Yo quiero dar la noticia
Yo quiero gritar que me quieres a los cuatro vientos
No quiero guardarme lo que siento
Gracias, gracias, gracias
Quiero agradecer a quien corresponda
Why a nosotros dos el premio a la audacia
Por entrar al mar por la parte más honda
Tiempo al tiempo al tiempo
Cada huella irá encontrando su arena
Cada beso deteniendo un momento
Why cada canción matando una pena
Yo quiero dar la noticia
Yo quiero gritar que me quieres a los cuatro vientos
No quiero guardarme lo que siento
Gracias, gracias, gracias
Gracias, gracias, gracias
Gracias, gracias, gracias
Gracias, gracias, gracias"""



In [53]:
print(df.to_string())

          id                                             lyrics                                     title                                                url
0    1427096  \n\nGira el haz de luz para que se vea desde a...                  12 Segundos De Oscuridad  https://genius.com/Jorge-drexler-12-segundos-d...
1    1276296  \n\nNo hay rincón en esta casa\nQue no te haga...                                  730 días   https://genius.com/Jorge-drexler-730-dias-lyrics
3    3204468  \n\n\n Letra de "Abracadabras" ft. Julieta Ven...                              Abracadabras  https://genius.com/Jorge-drexler-abracadabras-...
4    1719443  \n\n\nLetra de "Al Otro Lado del Río"Clavo mi ...                      Al Otro Lado del Río  https://genius.com/Jorge-drexler-al-otro-lado-...
5    1092085  \n\nMira\nYo aquí me bajo\nYo dejo el tren en ...                             Alto el fuego  https://genius.com/Jorge-drexler-alto-el-fuego...
6    1251195  \n\nAna\nQué no daría yo por ver un día\nTu 

#### Remove remaining duplicates, covers, and songs in other languages

In [82]:
df = df.drop(21) #Club Tonight
df = df.drop(26) #Dance, dance, dance
df = df.drop(28) #Dance me to the end of love
df = df.drop(37) #dom de iludir
df = df.drop(63) #fractura de escafoides tarsiano derecho
df = df.drop(74) #high and dry
df = df.drop(77) #i dont worry about a thing
df = df.drop(84) #la infidelidad de la era informatica, duplicada
df = df.drop(94) #lontano, lontano
df = df.drop(119) #o que sera, duplicada
df = df.drop(146) #stay
df = df.drop(156) #todo se transforma, duplicada
df = df.drop(158) #Toíto cái lo traigo andao
df = df.drop(174) #un pais con el nombre de un rio, duplicada


#### Removes excess newlines

In [83]:
df['lyrics'].replace(regex=True,inplace=True,to_replace=r'(\n)+',value=r'\n')
df['lyrics'].replace(regex=True,inplace=True,to_replace=r'^(\n)+',value=r'')

In [56]:
df.to_csv('data/raw_songs_v2.csv', sep=';',index=False)

#### Some manual corrections of lyrics

In [167]:
df = pd.read_csv('data/raw_songs_v3.tsv', sep='\t')
df

,id,lyrics,title,url
0,1427096,Gira el haz de luz para que se vea desde alta ...,12 Segundos De Oscuridad,https://genius.com/Jorge-drexler-12-segundos-d...
1,1276296,No hay rincón en esta casa Que no te haga regr...,730 días,https://genius.com/Jorge-drexler-730-dias-lyrics
2,3204468,Ponemos nota sobre nota Palabra sobre palabra ...,Abracadabras,https://genius.com/Jorge-drexler-abracadabras-...
3,1719443,Clavo mi remo en el agua Llevo tu remo en el m...,Al Otro Lado del Río,https://genius.com/Jorge-drexler-al-otro-lado-...
4,1092085,Mira Yo aquí me bajo Yo dejo el tren en esta e...,Alto el fuego,https://genius.com/Jorge-drexler-alto-el-fuego...
5,1251195,Ana Qué no daría yo por ver un día Tu ropa en ...,Ana,https://genius.com/Jorge-drexler-ana-lyrics
6,1331843,Anda Quítate el vestido Las flores y las tramp...,Anda,https://genius.com/Jorge-drexler-anda-lyrics
7,1375601,Antes de mi tu no eras tu Antes de ti yo no er...,Antes,https://genius.com/Jorge-drexler-antes-lyrics
8,1360422,Era el tiempo del cambio El tiempo de la estam...,Aquellos tiempos,https://genius.com/Jorge-drexler-aquellos-tiem...
9,1717480,Se es lo que se es Lo que siempre se ha sido S...,"Aquiles, por Su Talón Es Aquiles",https://genius.com/Jorge-drexler-aquiles-por-s...


## Let's add data manually to the db.
### Albums.
#### - If the song appears in more than one album, the one that counts is the first.
#### - If it doesn't appear in an official studio album, it will be 'Others'

In [168]:
dict_albums_songs = {
#     'La Luz Que Sabe Robar':[ # sin repetidos
#         'Bienvenida','La Luna De Espejos','Tatuaje','Equipaje',
#         'Edén','Un Solo Color','Vaivén','Macarena Y El Espejo',
#         'Luna Del Caño'
#     ],
#     'Vaivén':[ # sin repetidos
#         'Luna Negra', 'Tú',
#         '730 Días', 'Ganas De Ti',
#         'Zamba Del Olvido', 'Un Lugar En Tu Almohada',
#         'Dos Colores: Blanco Y Negro', 'Ana'
#     ],
#     'Radar':[ # sin repetidos
#         'Radar','No Te Creas' ,'Tu Voyeur','Caiguá','Cerca Del Mar','Gégé','Riéndose De Mí','Veneno',
#         'Era De Amar','El Valle De Las Leñas Amarillas',
#         'La Aparecida'
#     ],
    'La Luz Que Sabe Robar':[ # con repetidos
        'Bienvenida', 'La Luna De Espejos', 'Tatuaje', 
        'Equipaje', 'Eden', 'La Aparecida', 'Un Solo Color',
        'Vaivén', 'Macarena Y El Espejo', 'Luna Del Caño'
    ],
    'Radar': [ # con repetidos
        'Radar', 'No Te Creas', 'Tu Voyeur', 'Caiguá',
        'Cerca Del Mar', 'Gégé', 'Riéndose De Mí', 'Veneno', 
        'Era De Amar', 'Bienvenida', 'La Luna De Espejos',
        'Edén','La Aparecida', 'Vaivén'
    ],
    'Vaivén':[ # con repetidos
        'Luna Negra', 'Cerca Del Mar', 'Tú', 'Era De Amar',
        '730 Días', 'Ganas De Ti', 'Zamba del olvido',
        'Un Lugar En Tu Almohada', 'La Luna De Espejos',
        'Tu Voyeur', 'Dos Colores: Blanco Y Negro', 'Vaivén', 'Ana'
    ],
    'Llueve':[
        'De Amor Y De Casualidad','Antes','Llueve','Flores En El Mar',
        'Murga Reggae','No Pienses De Más','Cara B','La Huella De Tu Mirada',
        'Milonga Paraguaya','Tu Piel','Paisaje Lunar','Montevideo'
    ],
    'Frontera':[
        'La Edad Del Cielo','Memoria Del Cuero','Frontera','Rio Abajo',
        'Corazón De Cristal','Madre Tierra','Princesa Bacana','Alto El Fuego',
        'Aquellos Tiempos','El Sur Del Sur','Camino A La Paloma'
    ],
    'Sea':[
        'El Pianista Del Gueto De Varsovia','Sea','Causa Y Efecto',
        'Horas','Crece','Tamborero','Nada Menos','Me Haces Bien','Durante',
        'Raquel','Uno','Un País Con Nombre De Río'
    ],
    'Eco':[
        'Eco','Deseo','Todo Se Transforma',
        'Guitarra Y Vos','Transporte','Milonga Del Moro Judío',
        'Polvo De Estrellas ','Se Va, Se Va, Se Fue','Don De Fluir','Fusión','Salvapantallas'
    ],
    '12 Segundos de Oscuridad':[
        '12 Segundos De Oscuridad','La Vida Es Más Compleja De Lo Que Parece','Transoceánica',
        'Disneylandia','El Otro Engranaje','High And Dry','La Infidelidad En La Era Informática',
        'Hermana Duda','Inoportuna','Quien Quiera Que Seas','Soledad','Sanar'
    ],
    'Amar la trama':[
        'Tres Mil Millones De Latidos','La Trama Y El Desenlace','Las Transeúntes','La Nieve En La Bola De Nieve',
        'Mundo Abisal','Toque De Queda','Una Canción Me Trajo Hasta Aquí','Aquiles, Por Su Talón Es Aquiles',
        "I Don't Worry About A Thing",'Noctiluca','Todos A Sus Puestos','Telón'
    ],
    'Bailar en la cueva':[
        'Bailar En La Cueva', 'Bolivia', 'Data Data', 'La Luna De Rasquí', 'Universos Paralelos', 
        'Todo Cae', 'Esfera', 'La Plegaria Del Paparazzo', 'La Noche No Es Una Ciencia Exacta', 
        'El Triángulo De Las Bermudas', 'Organdí'
    ],
    'Salvavidas de hielo':[
        'Movimiento','Telefonía','Silencio','Pongamos Que Hablo De Martínez',
        'Estalactitas','Asilo','Abracadabras','Mandato','Despedir A Los Glaciares',
        'Quimera','Salvavidas De Hielo'
    ]
}

In [169]:
df['album'] = ''
df["duplicated"] = False
original_length = len(df)
for index, row in df.iterrows():
    if index > original_length - 1 :
        break
    for album, songs in dict_albums_songs.items():
        for a_song in songs:
            if row['title'].lower() == a_song.lower():
                next_row = len(df)
                if not df.at[index, 'album']:
                    df.at[index, 'album'] = album
                else:
                    df.loc[next_row] = df.loc[index]
                    df.at[next_row, 'album'] = album
                    df.at[next_row, 'duplicated'] = True



In [170]:
df

,id,lyrics,title,url,album,duplicated
0,1427096,Gira el haz de luz para que se vea desde alta ...,12 Segundos De Oscuridad,https://genius.com/Jorge-drexler-12-segundos-d...,12 Segundos de Oscuridad,False
1,1276296,No hay rincón en esta casa Que no te haga regr...,730 días,https://genius.com/Jorge-drexler-730-dias-lyrics,Vaivén,False
2,3204468,Ponemos nota sobre nota Palabra sobre palabra ...,Abracadabras,https://genius.com/Jorge-drexler-abracadabras-...,Salvavidas de hielo,False
3,1719443,Clavo mi remo en el agua Llevo tu remo en el m...,Al Otro Lado del Río,https://genius.com/Jorge-drexler-al-otro-lado-...,,False
4,1092085,Mira Yo aquí me bajo Yo dejo el tren en esta e...,Alto el fuego,https://genius.com/Jorge-drexler-alto-el-fuego...,Frontera,False
5,1251195,Ana Qué no daría yo por ver un día Tu ropa en ...,Ana,https://genius.com/Jorge-drexler-ana-lyrics,Vaivén,False
6,1331843,Anda Quítate el vestido Las flores y las tramp...,Anda,https://genius.com/Jorge-drexler-anda-lyrics,,False
7,1375601,Antes de mi tu no eras tu Antes de ti yo no er...,Antes,https://genius.com/Jorge-drexler-antes-lyrics,Llueve,False
8,1360422,Era el tiempo del cambio El tiempo de la estam...,Aquellos tiempos,https://genius.com/Jorge-drexler-aquellos-tiem...,Frontera,False
9,1717480,Se es lo que se es Lo que siempre se ha sido S...,"Aquiles, por Su Talón Es Aquiles",https://genius.com/Jorge-drexler-aquiles-por-s...,Amar la trama,False


In [171]:
df[df['album'].isnull()]

,id,lyrics,title,url,album,duplicated


In [172]:
df = df.drop(49) #found another duplicate, 'Gracias'
df = df.drop(6) #not an original song
df = df.drop(33) #Pablo Neruda's lyrics
df = df.drop(36) #not an original song
df = df.drop(68) #duplicate! luna del cabo
df = df.drop(76) #not an original song
df = df.drop(86) #Neruda's lyrics!
df = df.drop(87) #chico buarques
df = df.drop(122) #not drexler's lyrics
df = df.drop(131) #cover

In [173]:
df[df['album'].isnull()]
df.at[3, 'album'] = 'Others'
df.at[27, 'album'] = 'Vaivén'
df.at[28, 'album'] = 'Cara B'
df.at[32, 'album'] = '12 Segundos de Oscuridad'
df.at[35, 'album'] = 'Sea'

df.at[38, 'album'] = 'Bailar en la cueva'
df.at[38, 'title'] = 'El triángulo de las bermudas'

df.at[48, 'album'] = 'Cara B'
df.at[57, 'album'] = '12 Segundos de Oscuridad'
df.at[59, 'album'] = 'La Luz Que Sabe Robar'

df.at[62, 'album'] = 'Bailar en la cueva'
df.at[62, 'title'] = 'La noche no es una ciencia exacta'

df.at[66, 'album'] = 'Others'
df.at[88, 'album'] = 'Bailar en la cueva'
df.at[90, 'album'] = 'Eco'
df.at[93, 'album'] = 'Others'
df.at[99, 'album'] = 'Frontera'
df.at[126, 'album'] = 'Sea'
df.at[128, 'album'] = 'Others'


In [174]:
print(df.sort_values(['album','title']).to_string())

          id                                             lyrics                                 title                                                url                     album  duplicated
0    1427096  Gira el haz de luz para que se vea desde alta ...              12 Segundos De Oscuridad  https://genius.com/Jorge-drexler-12-segundos-d...  12 Segundos de Oscuridad       False
25   1143418  Hijo de inmigrantes rusos casado en Argentina ...                          Disneylandia  https://genius.com/Jorge-drexler-disneylandia-...  12 Segundos de Oscuridad       False
32   1340418  Todas las horas, todos los besos Cada recuerdo...             El Fuego Y El Combustible  https://genius.com/Jorge-drexler-el-fuego-y-el...  12 Segundos de Oscuridad       False
34   1250670  El deseo sigue un curso paralelo Y la historia...                     El Otro Engranaje  https://genius.com/Jorge-drexler-el-otro-engra...  12 Segundos de Oscuridad       False
51   1483931  No tengo a quien rezarle Pidien

### Add years

In [175]:
def label_year (row):
    if row['album'] == 'La Luz Que Sabe Robar':
        return 1992
    if row['album'] == 'Radar':
        return 1994
    if row['album'] == 'Vaivén':
        return 1996
    if row['album'] == 'Llueve':
        return 1997
    if row['album'] == 'Frontera':
        return 1999
    if row['album'] == 'Sea':
        return 2001
    if row['album'] == 'Eco':
        return 2004
    if row['album'] == '12 Segundos de Oscuridad':
        return 2006
    if row['album'] == 'Cara B':
        return 2008
    if row['album'] == 'Amar la trama':
        return 2010
    if row['album'] == 'Bailar en la cueva':
        return 2014
    if row['album'] == 'Salvavidas de hielo':
        return 2017
    return ''

In [176]:
df['year'] = df.apply(lambda row: label_year (row),axis=1)
df[df['album'] == 'Others']

,id,lyrics,title,url,album,duplicated,year
3,1719443,Clavo mi remo en el agua Llevo tu remo en el m...,Al Otro Lado del Río,https://genius.com/Jorge-drexler-al-otro-lado-...,Others,False,
66,3157514,Tu historia es tan antigua como las reglas del...,La Vida Entera,https://genius.com/Jorge-drexler-la-vida-enter...,Others,False,
93,2223097,Entrar en este verso como el viento Que mueve ...,Que el soneto nos tome por sorpresa,https://genius.com/Jorge-drexler-que-el-soneto...,Others,False,
128,3157510,Quién le roba un beso a Maracaná? Uruguay nomá...,Uruguay nomás,https://genius.com/Jorge-drexler-uruguay-nomas...,Others,False,


In [177]:
#Others
df.at[3, 'year'] = 2005
df.at[66, 'year'] = 2016
df.at[93, 'year'] = 2010
df.at[128, 'year'] = 2014


In [178]:
df = df.sort_values(['year','album']).reset_index(drop=True)

In [179]:
df

,id,lyrics,title,url,album,duplicated,year
0,1408064,El perfume de la sal Del cabo Santa María Reco...,Bienvenida,https://genius.com/Jorge-drexler-bienvenida-ly...,La Luz Que Sabe Robar,False,1992
1,1663095,Nos delata el equipaje Y la duda al caminar Su...,Equipaje,https://genius.com/Jorge-drexler-equipaje-lyrics,La Luz Que Sabe Robar,False,1992
2,1475055,Se va la tarde en Zanja Honda La playa es brum...,La aparecida,https://genius.com/Jorge-drexler-la-aparecida-...,La Luz Que Sabe Robar,False,1992
3,1737867,"Mabel, dejó El bolso con unas amigas Y salió a...",La luna de espejos,https://genius.com/Jorge-drexler-la-luna-de-es...,La Luz Que Sabe Robar,False,1992
4,2007190,Luna del Cabo Boca de túnel Lunar plateado Sob...,La luna del cabo,https://genius.com/Jorge-drexler-la-luna-del-c...,La Luz Que Sabe Robar,False,1992
5,887202,"Ella se pinta con un sylvapen Dibujándose, dib...",Macarena y el espejo,https://genius.com/Jorge-drexler-macarena-y-el...,La Luz Que Sabe Robar,False,1992
6,967545,Como un tatuaje desteñido Me iré borrando de t...,Tatuaje,https://genius.com/Jorge-drexler-tatuaje-lyrics,La Luz Que Sabe Robar,False,1992
7,1053846,Mírenla salir sobre la marea Nadie que la ve c...,Un solo color,https://genius.com/Jorge-drexler-un-solo-color...,La Luz Que Sabe Robar,False,1992
8,1618541,Tu caricia no me afecta Yo la puedo tolerar Si...,Vaivén,https://genius.com/Jorge-drexler-vaiven-lyrics,La Luz Que Sabe Robar,False,1992
9,871783,En una quinta De la calle Caiguá En un rincón ...,Caiguá,https://genius.com/Jorge-drexler-caigua-lyrics,Radar,False,1994


In [180]:
df.to_csv('data/drexler_complete_lyrics.csv', sep=';',index=False)

## Spotify

In [12]:
artist_name = 'Jorge Drexler'
client_credentials_manager = SpotifyClientCredentials(client_id='a9114ca9983341f4b5a3c70a8e1b4c78',
                                                      client_secret='1e8bac81838d4c968484b035d68f8b2b')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [14]:
artist_id = sp.search(q='artist:' + artist_name, type='artist')['artists']['items'][0]['id']
albums_ids = [(x['id'], x['name']) for x in sp.artist_albums(artist_id, album_type='album')['items']]
albums_ids = {x[1]: x[0] for x in albums_ids}

In [16]:
del albums_ids['Cara B']
del albums_ids['La Edad Del Cielo']

In [17]:
spotify_raw_df = pd.DataFrame()
for an_album_title, an_album_id in albums_ids.items():
    a_dataframe = pd.DataFrame(sp.album_tracks(an_album_id)['items'])
    a_dataframe['album'] = an_album_title
    if not spotify_raw_df.shape[0]:
        spotify_raw_df = pd.DataFrame(a_dataframe)
    else:
        spotify_raw_df = pd.concat([spotify_raw_df, a_dataframe])

In [19]:
del spotify_raw_df['artists']
del spotify_raw_df['available_markets']
del spotify_raw_df['explicit']
del spotify_raw_df['external_urls']
del spotify_raw_df['href']
del spotify_raw_df['is_local']
del spotify_raw_df['preview_url']
del spotify_raw_df['type']
spotify_raw_df = spotify_raw_df.reset_index()

In [22]:
spotify_raw_df

,index,disc_number,duration_ms,id,name,track_number,uri,album,duration_s
0,0,1,257360,43EVN2D60aYJf8PMuGuQH9,El Pianista Del Gueto De Varsovia,1,spotify:track:43EVN2D60aYJf8PMuGuQH9,Sea,257.36
1,1,1,225200,6z98uNlBjkisBUnv5zMQWE,Sea,2,spotify:track:6z98uNlBjkisBUnv5zMQWE,Sea,NaN
2,2,1,243760,4AnMHETBYUYoZeFJRNnE8n,Causa Y Efecto,3,spotify:track:4AnMHETBYUYoZeFJRNnE8n,Sea,NaN
3,3,1,218186,736D1R1c4aoGcn7ZEmtyH4,Horas,4,spotify:track:736D1R1c4aoGcn7ZEmtyH4,Sea,NaN
4,4,1,190600,5LgIR0Fzfc9vJy2Yx4WI0P,Crece,5,spotify:track:5LgIR0Fzfc9vJy2Yx4WI0P,Sea,NaN
5,5,1,264293,57ypE3GdN7ZLaZou7JAu9t,Tamborero,6,spotify:track:57ypE3GdN7ZLaZou7JAu9t,Sea,NaN
6,6,1,220413,68jCHd8ltpjWwj0cDX2G1y,Nada Menos,7,spotify:track:68jCHd8ltpjWwj0cDX2G1y,Sea,NaN
7,7,1,184546,4UKe38tm5NkRpuFCxwewEe,Me Haces Bien,8,spotify:track:4UKe38tm5NkRpuFCxwewEe,Sea,NaN
8,8,1,197506,1z38CbWAVCe8mJ9r4uWTAa,Durante,9,spotify:track:1z38CbWAVCe8mJ9r4uWTAa,Sea,NaN
9,9,1,191200,2ZjCVvo1K2xcbkw3AJGy9f,Raquel,10,spotify:track:2ZjCVvo1K2xcbkw3AJGy9f,Sea,NaN


In [32]:
for index, row in spotify_raw_df.iterrows():
    print(index)
    audio_analysis = sp.audio_analysis(row['id'])
    audio_features = sp.audio_features([row['id']])[0]
    spotify_raw_df.at[index, 'duration_s'] = audio_analysis['track']['duration']
    spotify_raw_df.at[index, 'acousticness'] = audio_features['acousticness']
    spotify_raw_df.at[index, 'danceability'] = audio_features['danceability']
    spotify_raw_df.at[index, 'energy'] = audio_features['energy']
    spotify_raw_df.at[index, 'key'] = audio_features['key']
    spotify_raw_df.at[index, 'tempo'] = audio_features['tempo']
    spotify_raw_df.at[index, 'time_signature'] = audio_features['time_signature']
    spotify_raw_df.at[index, 'valence'] = audio_features['valence']
    spotify_raw_df.at[index, 'mode'] = audio_features['mode']
    clear_output(wait=True)

125


In [33]:
spotify_raw_df

,index,disc_number,duration_ms,id,name,track_number,uri,album,duration_s,acousticness,danceability,energy,key,tempo,time_signature,valence,mode
0,0,1,257360,43EVN2D60aYJf8PMuGuQH9,El Pianista Del Gueto De Varsovia,1,spotify:track:43EVN2D60aYJf8PMuGuQH9,Sea,257.36000,0.15200,0.831,0.498000,9.0,146.592,4.0,0.4470,0.0
1,1,1,225200,6z98uNlBjkisBUnv5zMQWE,Sea,2,spotify:track:6z98uNlBjkisBUnv5zMQWE,Sea,225.20000,0.01380,0.618,0.684000,9.0,135.014,4.0,0.4650,0.0
2,2,1,243760,4AnMHETBYUYoZeFJRNnE8n,Causa Y Efecto,3,spotify:track:4AnMHETBYUYoZeFJRNnE8n,Sea,243.76000,0.11400,0.603,0.684000,4.0,80.635,4.0,0.9200,1.0
3,3,1,218186,736D1R1c4aoGcn7ZEmtyH4,Horas,4,spotify:track:736D1R1c4aoGcn7ZEmtyH4,Sea,218.18667,0.60200,0.703,0.709000,7.0,140.965,4.0,0.9240,1.0
4,4,1,190600,5LgIR0Fzfc9vJy2Yx4WI0P,Crece,5,spotify:track:5LgIR0Fzfc9vJy2Yx4WI0P,Sea,190.60000,0.46600,0.649,0.510000,5.0,174.931,3.0,0.4760,1.0
5,5,1,264293,57ypE3GdN7ZLaZou7JAu9t,Tamborero,6,spotify:track:57ypE3GdN7ZLaZou7JAu9t,Sea,264.29333,0.45100,0.833,0.579000,11.0,94.008,4.0,0.8190,0.0
6,6,1,220413,68jCHd8ltpjWwj0cDX2G1y,Nada Menos,7,spotify:track:68jCHd8ltpjWwj0cDX2G1y,Sea,220.41333,0.28300,0.695,0.514000,9.0,132.065,4.0,0.4390,1.0
7,7,1,184546,4UKe38tm5NkRpuFCxwewEe,Me Haces Bien,8,spotify:track:4UKe38tm5NkRpuFCxwewEe,Sea,184.54667,0.32700,0.404,0.607000,9.0,149.604,3.0,0.5760,1.0
8,8,1,197506,1z38CbWAVCe8mJ9r4uWTAa,Durante,9,spotify:track:1z38CbWAVCe8mJ9r4uWTAa,Sea,197.50667,0.00226,0.608,0.706000,1.0,177.914,4.0,0.5040,1.0
9,9,1,191200,2ZjCVvo1K2xcbkw3AJGy9f,Raquel,10,spotify:track:2ZjCVvo1K2xcbkw3AJGy9f,Sea,191.20000,0.35500,0.743,0.611000,9.0,137.986,4.0,0.4390,1.0


## Let's merge!

In [35]:
genius_df = pd.read_csv('data/drexler_complete_lyrics.csv', sep=';')
genius_df.head()

,id,lyrics,title,url,album,duplicated,year
0,1408064,El perfume de la sal Del cabo Santa María Reco...,Bienvenida,https://genius.com/Jorge-drexler-bienvenida-ly...,La Luz Que Sabe Robar,False,1992
1,1663095,Nos delata el equipaje Y la duda al caminar Su...,Equipaje,https://genius.com/Jorge-drexler-equipaje-lyrics,La Luz Que Sabe Robar,False,1992
2,1475055,Se va la tarde en Zanja Honda La playa es brum...,La aparecida,https://genius.com/Jorge-drexler-la-aparecida-...,La Luz Que Sabe Robar,False,1992
3,1737867,"Mabel, dejó El bolso con unas amigas Y salió a...",La luna de espejos,https://genius.com/Jorge-drexler-la-luna-de-es...,La Luz Que Sabe Robar,False,1992
4,2007190,Luna del Cabo Boca de túnel Lunar plateado Sob...,La luna del cabo,https://genius.com/Jorge-drexler-la-luna-del-c...,La Luz Que Sabe Robar,False,1992


In [37]:
print(genius_df.shape)
print(spotify_raw_df.shape)

(132, 7)
(126, 17)


# Drop extra albums

In [50]:
genius_df = genius_df.drop(genius_df[genius_df['album'] == "Cara B"].index)
genius_df = genius_df.drop(genius_df[genius_df['album'] == "Others"].index)

In [52]:
print(genius_df.shape)
print(spotify_raw_df.shape)

(126, 7)
(126, 17)


In [62]:
spotify_raw_df[(spotify_raw_df['album'] == 'Sus primeras grabaciones 1992-1994 (La luz que sabe robar- Radar)') &
              (spotify_raw_df['disc_number'] == 1)]

,index,disc_number,duration_ms,id,name,track_number,uri,album,duration_s,acousticness,danceability,energy,key,tempo,time_signature,valence,mode
71,0,1,297973,3bgEz3NsFjabfbPN6DdOWN,Bienvenida,1,spotify:track:3bgEz3NsFjabfbPN6DdOWN,Sus primeras grabaciones 1992-1994 (La luz que...,297.97333,0.540,0.789,0.532,0.0,112.732,4.0,0.904,1.0
72,1,1,221893,6k3fTbDyuULy9kYA7OShyK,La luna de espejos,2,spotify:track:6k3fTbDyuULy9kYA7OShyK,Sus primeras grabaciones 1992-1994 (La luz que...,221.89333,0.930,0.712,0.225,5.0,87.520,4.0,0.429,1.0
73,2,1,301413,4ZejXkTaVyw3KffEcS86ix,Tatuaje,3,spotify:track:4ZejXkTaVyw3KffEcS86ix,Sus primeras grabaciones 1992-1994 (La luz que...,301.41333,0.600,0.532,0.571,9.0,79.529,4.0,0.215,1.0
74,3,1,239360,3N9qP0bV95JnvDvicZCIKQ,Equipaje,4,spotify:track:3N9qP0bV95JnvDvicZCIKQ,Sus primeras grabaciones 1992-1994 (La luz que...,239.36000,0.928,0.785,0.393,4.0,126.776,4.0,0.333,1.0
75,4,1,177946,47uN75sZhcmhu947dzPkuo,Eden,5,spotify:track:47uN75sZhcmhu947dzPkuo,Sus primeras grabaciones 1992-1994 (La luz que...,177.94667,0.921,0.643,0.311,6.0,137.278,4.0,0.773,1.0
76,5,1,276906,2Z3IaUbfcS8oqxmipQTkGh,La aparecida,6,spotify:track:2Z3IaUbfcS8oqxmipQTkGh,Sus primeras grabaciones 1992-1994 (La luz que...,276.90667,0.853,0.539,0.372,7.0,147.078,3.0,0.313,1.0
77,6,1,220266,0Rdr3kKgd5I9XEpBfv4b2k,Un solo color,7,spotify:track:0Rdr3kKgd5I9XEpBfv4b2k,Sus primeras grabaciones 1992-1994 (La luz que...,220.26667,0.793,0.560,0.484,9.0,136.740,4.0,0.610,0.0
78,7,1,170693,13wF870pnYuvSdrCHnqyvf,Vaiven,8,spotify:track:13wF870pnYuvSdrCHnqyvf,Sus primeras grabaciones 1992-1994 (La luz que...,170.69333,0.902,0.696,0.388,4.0,155.018,3.0,0.539,1.0
79,8,1,266666,24KGYSlfrvBE5vkrANLNc4,Macarena y el espejo,9,spotify:track:24KGYSlfrvBE5vkrANLNc4,Sus primeras grabaciones 1992-1994 (La luz que...,266.66667,0.960,0.416,0.242,0.0,161.452,4.0,0.214,0.0
80,9,1,242293,6EKhj17vbiLO1GqkC3vj7l,La luna del cabo,10,spotify:track:6EKhj17vbiLO1GqkC3vj7l,Sus primeras grabaciones 1992-1994 (La luz que...,242.29333,0.629,0.629,0.586,7.0,114.800,4.0,0.168,1.0


In [67]:
genius_df.to_csv('drexler_genius.csv', sep=';',index=False)
spotify_raw_df.to_csv('drexler_spotify.csv', sep=';',index=False, encoding='utf-8')